# Clustering Crypto

In [86]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [87]:
# Load the crypto_data.csv dataset.
file = "crypto_data.csv"
crypto_df = pd.read_csv(file)
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [88]:
# count the values being traded and not
print(crypto_df["IsTrading"].value_counts())

True     1144
False     108
Name: IsTrading, dtype: int64


In [89]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df.loc[(crypto_df["IsTrading"]==True)]

# confirm False values are removed
# print(crypto_df["IsTrading"].value_counts())
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [90]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df = crypto_df.dropna()

# confirm NaN values have been omitted
# crypto_df.count()

In [91]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns=["IsTrading"])
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [92]:
# Remove rows that have at least 1 null value.
# confirm that null values have been dropped
crypto_df.isnull().sum()

Unnamed: 0         0
CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
TotalCoinSupply    0
dtype: int64

In [93]:
# Keep the rows where coins are mined.
crypto_df = crypto_df.loc[(crypto_df["TotalCoinsMined"] != 0)]
crypto_df

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...,...
1238,ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [94]:
# Set the "Unnamed: 0" column as the index and remove the title
crypto_df.set_index("Unnamed: 0", inplace=True)
crypto_df.index.name = None
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [95]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_name_df = crypto_df[["CoinName"]]
coin_name_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [96]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns="CoinName")
crypto_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [97]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns=["Algorithm","ProofType"])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
# Standardize the data with StandardScaler().
data_scaler = StandardScaler()
scaled_df = data_scaler.fit_transform(X)
scaled_df

array([[-0.11674788, -0.15286468, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.09358885, -0.14499604, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [ 0.52587231,  4.4937636 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       ...,
       [-0.09523411, -0.13215444, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11658774, -0.15255408, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11674507, -0.15284989, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [99]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(scaled_df)
crypto_pca

array([[-0.33136251,  1.04813763, -0.52249195],
       [-0.31463422,  1.04793826, -0.52268657],
       [ 2.34222889,  1.52981259, -0.64199372],
       ...,
       [ 0.33208612, -2.25880565,  0.41715201],
       [-0.18777258, -1.99473883,  0.40037137],
       [-0.29493448,  0.79200051, -0.24322727]])

In [100]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data=crypto_pca, index=crypto_df.index, columns=["PC 1","PC 2","PC 3"])
pcs_df

,PC 1,PC 2,PC 3
42,-0.331363,1.048138,-0.522492
404,-0.314634,1.047938,-0.522687
1337,2.342229,1.529813,-0.641994
BTC,-0.130544,-1.296068,0.145681
ETH,-0.150898,-2.052672,0.409994
...,...,...,...
ZEPH,2.457924,0.946298,0.150228
GAP,-0.329399,1.047988,-0.522505
BDX,0.332086,-2.258806,0.417152
ZEN,-0.187773,-1.994739,0.400371


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [101]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1,11))

# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    
# define a df to plot elbow curve using hvplot
elbow_data = {"k": k, "inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

C:\Users\ssteffen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1039: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [102]:
# Initialize the K-Means model.
k = 4
model = KMeans(n_clusters=k, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

array([2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2,
       0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0,
       0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 0, 2,
       2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 2, 0,
       2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2,
       0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2,
       2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2,
       2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2,
       2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2,
       2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2,
       2, 2, 0, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2, 0,
       2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2,

In [103]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, pcs_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.concat([clustered_df, coin_name_df], axis=1) 

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
predictions_df = pd.DataFrame(predictions, index=crypto_df.index, columns=["Class"]) 
clustered_df = pd.concat([clustered_df, predictions_df], axis=1)

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.331363,1.048138,-0.522492,42 Coin,2
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.314634,1.047938,-0.522687,404Coin,2
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.342229,1.529813,-0.641994,EliteCoin,2
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.130544,-1.296068,0.145681,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.150898,-2.052672,0.409994,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.170958,-1.061385,0.032622,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.389645,1.155348,-0.490877,Dash,2
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.146021,-2.183844,0.449982,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.149333,-2.052799,0.409984,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.187772,-1.994739,0.400371,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [104]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    hover_data=["Algorithm"],
    hover_name="CoinName",
    width=800)

fig.update_layout(legend=dict(x=0, y=1), title="Cryptocurrency PCA Analysis")
fig.show()

In [105]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName","Algorithm","ProofType","TotalCoinSupply","TotalCoinsMined","Class"], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [110]:
# Print the total number of tradable cryptocurrencies.
# NOTE: It's not clear what is meant by "tradable" here.
crypto_count = clustered_df["CoinName"].count()
depleted_amount = clustered_df["TotalCoinSupply"].loc[(clustered_df["TotalCoinSupply"] == 0)].count()
print(f"There are {crypto_count} distinct cryptocurrencies in the dataset.")
print(f"{depleted_amount} of those cryptocurrencies are depleted.")

There are 533 distinct cryptocurrencies in the dataset.
35 of those cryptocurrencies are depleted.


In [107]:
#convert datatypes
clustered_df["TotalCoinSupply"] = clustered_df["TotalCoinSupply"].astype(float)
clustered_df["TotalCoinsMined"] = clustered_df["TotalCoinsMined"].astype(float)

# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
scaled_data

array([[4.20000000e-11, 5.94230127e-03],
       [5.32000000e-04, 7.00182308e-03],
       [3.14159265e-01, 3.53420682e-02],
       ...,
       [1.40022261e-03, 6.92655266e-03],
       [2.10000000e-05, 5.94962775e-03],
       [1.00000000e-06, 5.94243008e-03]])

In [108]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(scaled_data, index=clustered_df.index, columns=["TotalCoinSupply","TotalCoinsMined"])

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df = pd.concat([plot_df, clustered_df["CoinName"]], axis=1)

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df = pd.concat([plot_df, clustered_df["Class"]], axis=1)

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.005942,42 Coin,2
404,5.320000e-04,0.007002,404Coin,2
1337,3.141593e-01,0.035342,EliteCoin,2
BTC,2.100000e-05,0.005960,Bitcoin,0
ETH,0.000000e+00,0.006050,Ethereum,0
LTC,8.400000e-05,0.006006,Litecoin,0
DASH,2.200000e-05,0.005951,Dash,2
XMR,0.000000e+00,0.005960,Monero,0
ETC,2.100000e-04,0.006056,Ethereum Classic,0
ZEC,2.100000e-05,0.005950,ZCash,0


In [109]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    by="Class",
    hover_cols=["CoinName"]
)


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)